# Homework 5


For a simple counting experiment, the expected background event is $b$ and the observed event is $n$. 
The best estimator for signal event $s$ is:
$$s=n-b.$$

There are different metrics to evaluate discovery significance.

* Simplified Z0 
$$ Z_{0, simple} = s/\sqrt{b}$$

* Asymptotic Z0
$ Z_{0, asymptotic} = \sqrt{2((s+b) \ \mathrm{ln} (1+s/b)-s)}$

* Bayesian Z0
$ p-value = \int_{n}^{\infty}\mathrm{Poisson}(k|b) dk$.
$Z_{0, Bayesian} =  \mathrm{Z score \ 1-tailed \ (p-value)} $

* Frequentist Z0
$ p-value =  \int_{q_{0,n}}^{\infty} f(q_0|b) dq_0 $
$Z_{0, Frequentist} = \mathrm{Z score \ 1-tailed \ (p-value)} $


In this exercise, we will implement each of the metrics and compare consistency.

In [69]:
import numpy as np
import matplotlib.pyplot as plt
import iminuit.minimize as minimize
from tqdm import tqdm
import scipy
import math
from scipy.stats import poisson
from scipy.stats import norm

# Define test statistics q_0 for Frequentist approach
# 1. We require signal event s > 0 for positive signal yield.
#    Therefore, the test statistics for q_0 are 0 if N_obs <= Nb.
# 2. Compute two Poisson log likelihoods for:
#    a) background only model
#    b) signal + background model
#    Evaluate -2 log likelihood ratio between a) and b)
def q0(N_obs,Nb):
    # Write your code here
    s = N_obs - Nb

    # Ensure the test statistic is 0 if s <= 0.
    if (s <= 0):
        q0_out = 0
    else:
        # Use logpmf to prevent loss of precision due to rounding.
        L1 = poisson.logpmf(N_obs, Nb, loc=0)
        L2 = poisson.logpmf(N_obs, s + Nb, loc=0)

        # print("L1: %f" % L1)
        # print("L2: %f" % L2)

        # Only need to subtract 2 likelihood values since we already took the log.
        q0_out = -2 * (L1 - L2)

    return q0_out

Implement four metrics:

In [82]:
def SimplifiedZ0(N_obs, N_b):
    # Write your code here

    s = N_obs - N_b
    Zscore = s / np.sqrt(N_b)

    return Zscore

def AsymptoticZ0(N_obs, N_b):
    # Write your code here

    s = N_obs - N_b
    Zscore = np.sqrt(2*((s + N_b) * np.log(1 + s / N_b) - s))

    return Zscore

def BayesianZ0(N_obs, N_b):
    # Write your code here

    s = N_obs - N_b
    pvalue = 1 - poisson.cdf(N_obs, N_b)
    Zscore = norm.ppf(1 - pvalue)

    return Zscore

def FrequentistZ0(N_obs, N_b):
    # Write your code here
    #set random seed to guarantee reproducibility
    np.random.seed(seed=8)

    # Step1. Generate f(q_0|B) distribution
    
    s = N_obs - N_b
    # Generate 10K toy experiments based on signal and background hypothesis
    # Perform discovery test on each toy experiment to obtain q_0 distribtuion
    Ntoys = 10000
    q0s = np.zeros(Ntoys)
    for i in tqdm(range(Ntoys)):
        toy_hist = np.random.poisson(N_b)
        q0_out = q0(toy_hist, N_b)
        q0s[i] = (q0_out)


    # Step2. Calculate q_{0,obs} of the observed event.
    q0_obs = q0(N_obs, N_b)

    # Step3. Compute p-value, given by the fraction of toy experiments with q_0 greater or equal to q_{0,obs}
    pvalue = len(q0s[q0s > q0_obs]) / len(q0s)
    Zscore = norm.ppf(1 - pvalue)
    
    return Zscore

Now, let's apply our code for numerical calculations.

Consider the case that observed events n=5.

There are 4 different background models:

* case 1: b=0.5
* case 2: b=1
* case 3: b=3
* case 4: b=5
* case 5: b=10

Calculate the discovery significance for each of the background models, respectively.

In [83]:
# Write your code here
N_obs = 5
Nb_list = [0.5, 1, 3, 5, 10]
for N_b in Nb_list:
    # Write your code here
    print("SimplifiedZ0: %.2f" % SimplifiedZ0(N_obs, N_b))
    print("AsymptoticZ0: %.2f" % AsymptoticZ0(N_obs, N_b))
    print("BayesianZ0: %.2f" % BayesianZ0(N_obs, N_b))
    print("FrequentistZ0: %.2f" % FrequentistZ0(N_obs, N_b))

    # q0_out = q0(N_obs, N_b)
    # print(q0_out)
    print("\n")

SimplifiedZ0: 6.36
AsymptoticZ0: 3.75
BayesianZ0: 4.19


100%|██████████| 10000/10000 [00:00<00:00, 10080.58it/s]


FrequentistZ0: 3.72


SimplifiedZ0: 4.00
AsymptoticZ0: 2.85
BayesianZ0: 3.24


100%|██████████| 10000/10000 [00:00<00:00, 18179.99it/s]


FrequentistZ0: 3.29


SimplifiedZ0: 1.15
AsymptoticZ0: 1.05
BayesianZ0: 1.38


...

FrequentistZ0: 1.36


SimplifiedZ0: 0.00
AsymptoticZ0: 0.00
BayesianZ0: 0.29


100%|██████████| 10000/10000 [00:00<00:00, 13812.20it/s]


FrequentistZ0: 0.33


SimplifiedZ0: -1.58
AsymptoticZ0: 1.75
BayesianZ0: -1.50


100%|██████████| 10000/10000 [00:00<00:00, 12922.70it/s]

FrequentistZ0: 0.22




Describe the consistency between different metrics for each background model case.

Write your answers here:


As expected, across all metrics, the significance for a given $N_{obs}$ decreases as the background signal $N_b$ increases. Note, however, that the z-scores given by our 4 methods often give fairly different results. In the first result, SimplifiedZ0 gives the greatest z-score, followed by the BayesianZ0, then by AsymptoticZ0, and finally by the frequentist z-score. The order of these z-scores seems to change as the background signal increases, though.